In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import requests
def get_reuters_articles(URL):

    chrome_options = Options()
    #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    articles=soup.find_all("div", class_=lambda c: c and c.startswith("story-card__area-headline"))
    news_data = []
    for article in articles:
        title = article.get_text(strip=True)
        link = article.find('a', href=True)['href']
        if not link.startswith('http'):
            link = base_URL + link
        data_point = {'offset': offset_nb,'section':section_val,'Date': title, 'link': link}
        news_data.append(data_point)
    return pd.DataFrame(news_data)

get_article_sets =set()
for section_val in ['markets']:
    offset_nb = 0
    base_URL="https://www.reuters.com"
    search_query="/site-search/?query=gold"
    offset =f"&offset={offset_nb}"
    section=f"&section={section_val}"
    URL = base_URL + search_query + offset + section
    try:
        df = get_reuters_articles(URL)
        get_article_sets.add((section_val ,offset_nb))
    except Exception as e:
        print(f"Error fetching articles from {URL}: {e}")
    for offset_nb in range(20, 1000, 20):
        base_URL="https://www.reuters.com"
        search_query="/site-search/?query=gold"
        offset =f"&offset={offset_nb}"
        section=f"&section={section_val}"
        URL = base_URL + search_query + offset + section
        try:
            df_latest=get_reuters_articles(URL)
            df = pd.concat([df, df_latest], ignore_index=True)
            get_article_sets.add((section_val ,offset_nb))
        except Exception as e:
            print(f"Error fetching articles from {URL}: {e}")
            continue

In [36]:
def get_article_text(link):
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    time.sleep(2)
    page_soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    article = page_soup.find_all("div", class_=lambda c: c and c.startswith("article-body__paragraph"))
    article_text = ""
    for para in article:
        paragraph_text = para.get_text(strip=True)
        article_text = article_text + "." + paragraph_text
    date = page_soup.find("span", class_=lambda c: c and c.startswith("date-line__date")).get_text(strip=True)
    return article_text, date

In [27]:
df = pd.read_csv('../reuters_articles.csv')

In [40]:
#pop each row from df and get the article text
df['Article_Text'] = None
df['Date'] = None
for index, row in df.iterrows():
    if pd.notna(row['Article_Text']):
        continue  # Skip if Article_Text is already populated
    try:
        article_text, date = get_article_text(row['link'])
    except Exception as e:
        print(f"Error fetching article text for {row['link']}: {e}")
        continue
    df.at[index, 'Article_Text'] = article_text
    df.at[index, 'Date'] = date
    #print the df at the index
    print(df.iloc[index])
    

offset                                                          0
section                                                   markets
Title           When it comes to a US debt default, never say ...
link            https://www.reuters.com/markets/when-it-comes-...
Article_Text    .NEW YORK, June 11 - “The United States of Ame...
Date                                                June 11, 2025
Name: 0, dtype: object
offset                                                          0
section                                                   markets
Title           Wall Street closes with modest gains, dollar w...
link            https://www.reuters.com/markets/global-markets...
Article_Text    .NEW YORK, June 2 (Reuters) - Wall Street ende...
Date                                                 June 3, 2025
Name: 1, dtype: object
offset                                                          0
section                                                   markets
Title           TSX's gains se

In [44]:
#drop offset and section columns
df_2 = df.drop(columns=['offset', 'section','link'])
df_2['Content'] = "Tittle:"+df_2['Title'] + '\nArticle:' + df_2['Article_Text']
df_2=df_2.drop(columns=['Title', 'Article_Text'])
df_2['Date'] = pd.to_datetime(df_2['Date'], errors='coerce')
#save the dataframe to a csv file
df_2.to_csv('../reuters_gold_articles_processed.csv', index=False)